In [ ]:
from sklearn.model_selection import train_test_split
import re
import sys
from sklearn.externals import  joblib
from sklearn.model_selection import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
import json
import nltk
import pyaudio
from konlpy.tag import *
import serial
import pyaudio
import wave
import time
import urllib3
import json
import base64
kkma=Kkma()
tw=Twitter()
po = pyaudio.PyAudio()
for index in range(po.get_device_count()): 
    desc = po.get_device_info_by_index(index)

In [ ]:
def ngram(command):
    temp=command+' '
    for i in range(1,4):
        for j in range(len(command)-i+1):
            temp=temp+command[j:j+i]+' '
    return temp

class textList:
    def __init__(self):
        self.command=[]
        self.answer=[]
    def append(self,text,text2):
        self.command.append(text)
        self.answer.append(int(text2))
        
    def nouns(self):
        nouns_list=[]
        for nowC, nowA in zip(self.command, self.answer):
            tmp=''
            arr=tw.pos(nowC)
            arr2=kkma.pos(nowC)
            for pos, pos2 in zip(arr, arr2):
                if pos[1] == 'Noun' or pos[1] == 'Verb':
                    tmp=tmp+ngram(pos[0]) + ' '
                if pos2[1][0] =='N' or pos[1][0] == 'V':
                    tmp=tmp+ngram(pos2[0])+' '
            nouns_list.append((tmp, nowA))
        return nouns_list

In [ ]:

#print(ser.portstr) #연결된 포트 확인.

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 3
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("Start to record the audio.")
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Recording is finished.")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()
openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
accessKey = ""
audioFilePath = "output.wav"
languageCode = "korean"

file = open(audioFilePath, "rb")
audioContents = base64.b64encode(file.read()).decode("utf8")
file.close()

requestJson = {
    "access_key": accessKey,
    "argument": {
        "language_code": languageCode,
        "audio": audioContents
    }
}

http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)

jsonData=json.loads(response.data.decode('utf-8'))
resultT=jsonData['return_object']['recognized']
print(resultT)
x_text_list=[]
f = open("x_text_list.out", 'r')
lines=f.readlines()
for line in lines:
    x_text_list.append(line.strip('\n'))
gsvc=joblib.load('model.pkl')
vectorizer = CountVectorizer()
vectorizer.fit_transform(x_text_list)

testVoice = textList()
testVoice.append(resultT, 0)
predict=[]
predict= list(map(lambda i : i[0], testVoice.nouns()))
Result = gsvc.predict(vectorizer.transform(predict))
print(Result[0])
send='!'+str(Result[0])
ser = serial.Serial(
    port='COM4',\
    baudrate=9600,\
    parity=serial.PARITY_NONE,\
    stopbits=serial.STOPBITS_ONE,\
    bytesize=serial.EIGHTBITS,\
        timeout=0)
ser.write(bytes(send, encoding='ascii')) #출력방식1
time.sleep(3)
ser.close()